In [3]:
import pandas as pd
from auxiliary.paths import input_path, raw_path
from astropy.table import Table
import numpy as np
from auxiliary.columns import wise, galex, create_colors, calculate_colors, splus, error_splus
from auxiliary.correct_extinction import correction
import numpy as np

In [4]:
filename = "/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_dual_VAC_features.fits"
table = Table.read(os.path.join(input_path, filename))
table = table.to_pandas()
table["ID"] = table["ID"].str.decode('utf-8') 

In [5]:
table['W1_MAG'] = 22.5 - 2.5*np.log10(table['FW1']) # + 2.699
table['W2_MAG'] = 22.5 - 2.5*np.log10(table['FW2']) # + 3.339

# http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/namespacelsst_1_1afw_1_1image.html#a0a023f269211d52086723788764d484e
# return std::abs(fluxErr / (-0.4 * flux * std::log(10)));
table['e_W1_MAG'] = abs(table['e_FW1'] / (-0.4*table['FW1']*np.log(10)))
table['e_W2_MAG'] = abs(table['e_FW2'] / (-0.4*table['FW2']*np.log(10)))

table['W1_MAG'].replace([np.inf, -np.inf], np.nan, inplace=True)
table['W2_MAG'].replace([np.inf, -np.inf], np.nan, inplace=True)

/Users/Mariko/opt/anaconda3/envs/photoz/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/Mariko/opt/anaconda3/envs/photoz/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
table[wise+galex] = table[wise+galex].fillna(value=99)

In [8]:
correct = False

if correct:
    table = correction(table)
calculate_colors(table, broad=True, narrow=True, wise=True, galex = True, aper="PStotal")
features = create_colors(broad=True, narrow=True, wise=True, galex=True, aper="PStotal")
pos_splus = ["ID", "RA", "DEC"]

if correct:
    table[pos_splus+features+splus+wise+galex+error_splus].to_csv("/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_QSOz_VAC_ext.csv", index=False)
else:
    table[pos_splus+features+splus+wise+galex+error_splus].to_csv("/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_QSOz_VAC.csv", index=False)

In [12]:
corrected_df.dropna(subset=splus)
corrected_df.query(f"ID=='iDR4_3_STRIPE82-0170_0022523'")[splus]

,u_PStotal,g_PStotal,r_PStotal,i_PStotal,z_PStotal,J0378_PStotal,J0395_PStotal,J0410_PStotal,J0430_PStotal,J0515_PStotal,J0660_PStotal,J0861_PStotal
22522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
